In [1]:
import pandas as pd
import itertools
import random
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('../data/train_raw.csv')
print(train.shape)

(644, 9)


In [3]:
reshaped_df = pd.DataFrame({
    '질문': pd.concat([train['질문_1']] * 5 + [train['질문_2']] * 5).reset_index(drop=True),
    '답변': pd.concat([train['답변_1'], train['답변_2'] ,train['답변_3'], train['답변_4'],train['답변_5']]*2).reset_index(drop=True)
})
reshaped_df.shape

(6440, 2)

In [4]:
# 같은 카테고리 안에서 질문 2개&그에 따른 답변 2개씩 조합
setlist = []

random.seed(0)
qs = ['질문_1', '질문_2']
categories = train['category'].unique()

# 1. 카테고리별로 질문 2개씩 조합
for category in categories:
    category_idx = train[train['category'] == category]['id']
    # 2. row 2개씩 조합
    question_combinations = list(itertools.combinations(category_idx, 2))
    lens = len(question_combinations)

    for idx, (i,j) in enumerate(question_combinations):
        # 3. 질문 2개에 대한 답변 2개씩 조합
        for qi1, qi2 in itertools.product(qs, qs):
            q1 = train[train['id'] == i][qi1].values[0]
            q2 = train[train['id'] == j][qi2].values[0]
            a1 = train[train['id'] == i]['답변_2'].values[0]
            a2 = train[train['id'] == j]['답변_2'].values[0]
            setlist.append(pd.DataFrame({'질문': [q1+q2], '답변': [a1+a2]}))
        
        
trainset = pd.concat(setlist).reset_index(drop=True)
trainset.shape

(212960, 2)

In [5]:
dataset_v7 = pd.concat([reshaped_df, trainset])
print(dataset_v7.shape)

(219400, 2)


In [6]:
dataset_v7.head()

,질문,답변
0,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.


In [7]:
from sklearn.model_selection import train_test_split
train_ratio = 0.95  # 예시로 95%를 train 데이터로 사용
train_df, dev_df = train_test_split(dataset_v7, test_size=1-train_ratio, random_state=42)
print(train_df.shape)

(208429, 2)


In [8]:
train_df.to_csv('../data/train_v7.csv')
dev_df.to_csv('../data/eval_v7.csv')